# Basic Examples for Model with Metadata


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


# 1. Directly define in model's method

In [3]:
!pygmentize example-01/Model.py

import logging
import json
import os


class Model:
    def predict(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return features.tolist()

    def init_metadata(self):
        logging.info("metadata method  called")

        meta = {
            "name": "my-model-name",
            "versions": ["my-model-version-01"],
            "platform": "seldon-custom",
            "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1]}],
            "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}],
        }

        return meta


In [4]:
!pygmentize example-01/model_rest.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metadata-01_rest:0.1
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
    graph:
      children: []
      endpoint:
        type: REST
      name: my-model
      type: MODEL
    name: example
    replicas: 1


In [5]:
%%bash

kubectl apply -f example-01/model_rest.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model created
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [15]:
!curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model | jq .

{
  "inputs": [
    {
      "datatype": "BYTES",
      "name": "input",
      "shape": [
        1
      ]
    }
  ],
  "name": "my-model-name",
  "outputs": [
    {
      "datatype": "BYTES",
      "name": "output",
      "shape": [
        1
      ]
    }
  ],
  "platform": "seldon-custom",
  "versions": [
    "my-model-version-01"
  ]
}


In [18]:
import json

meta = !curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model 
meta = json.loads(meta[0])

assert meta == {
    "inputs": [{"datatype": "BYTES", "name": "input", "shape": [1]}],
    "name": "my-model-name",
    "outputs": [{"datatype": "BYTES", "name": "output", "shape": [1]}],
    "platform": "seldon-custom",
    "versions": ["my-model-version-01"],
}


In [19]:
!kubectl delete -f example-01/model_rest.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


# 2. Via environmental variable

In [20]:
!pygmentize example-02/Model.py

import logging
import json
import os


class Model:
    def predict(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        return features.tolist()


In [21]:
!pygmentize example-02/model_rest.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metadata-02_rest:0.1
          name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
          - name: MODEL_METADATA
            value: |
              {"name": "my-model-name",
               "versions": ["my-model-version-02"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [1]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [1]}]}
    graph:
      children: []
      endpoint:
        type: REST
      name: my-model
      type: MODEL
    name: example
    replicas: 1


In [22]:
%%bash

kubectl apply -f example-02/model_rest.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model created
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [23]:
!curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model | jq .

In [26]:
import json

meta = !curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model 
meta = json.loads(meta[0])

assert meta == {
    "inputs": [{"datatype": "BYTES", "name": "input", "shape": [1]}],
    "name": "my-model-name",
    "outputs": [{"datatype": "BYTES", "name": "output", "shape": [1]}],
    "platform": "seldon-custom",
    "versions": ["my-model-version-02"],
}


In [27]:
!kubectl delete -f example-02/model_rest.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


# 3. Prepackaged model servers: SKLearn Server

In [28]:
!pygmentize sklearn.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  annotations:
    seldon.io/executor: "true"
  name: iris
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: my-model
          env:
          - name: SELDON_LOG_LEVEL
            value: DEBUG
          - name: MODEL_METADATA
            value: |
              {"name": "my-model-name",
               "versions": ["my-sklearn-server"],
               "platform": "seldon-custom",
               "inputs": [{"name": "input", "datatype": "BYTES", "shape": [4]}],
               "outputs": [{"name": "output", "datatype": "BYTES", "shape": [3]}]}
    graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: my-model
    name: example
    replicas: 1


In [29]:
%%bash

kubectl apply -f sklearn.yaml

dep_name=$(kubectl get deploy -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')
kubectl rollout status deploy/${dep_name}

seldondeployment.machinelearning.seldon.io/seldon-model created
Waiting for deployment "seldon-model-example-0-my-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-my-model" successfully rolled out


In [31]:
!curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model | jq .

{
  "inputs": [
    {
      "datatype": "BYTES",
      "name": "input",
      "shape": [
        4
      ]
    }
  ],
  "name": "my-model-name",
  "outputs": [
    {
      "datatype": "BYTES",
      "name": "output",
      "shape": [
        3
      ]
    }
  ],
  "platform": "seldon-custom",
  "versions": [
    "my-sklearn-server"
  ]
}


In [32]:
import json

meta = !curl -s http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/metadata/my-model 
meta = json.loads(meta[0])

assert meta == {
    "inputs": [{"datatype": "BYTES", "name": "input", "shape": [4]}],
    "name": "my-model-name",
    "outputs": [{"datatype": "BYTES", "name": "output", "shape": [3]}],
    "platform": "seldon-custom",
    "versions": ["my-sklearn-server"],
}


In [33]:
!kubectl delete -f sklearn.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
